In [1]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Sun Oct  3 10:09:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

Then, mount your Drive to the Colab notebook: 

In [2]:
!rm -r sample_data

In [3]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
#drive.mount(str(drive_path))

In [4]:
stylegan2_repo_url  = 'https://github.com/dvschultz/stylegan2-ada' # or https://github.com/NVlabs/stylegan2-ada
project_path        = content_path / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Download StyleGAN2-ada

#!git config --global user.name "ArthurFDLR"
#!git config --global user.email "arthfind@gmail.com"
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
###
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}

/content/StyleGAN2-ADA
Install StyleGAN2-ADA
Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 364, done.
remote: Total 364 (delta 0), reused 0 (delta 0), pack-reused 364
Receiving objects: 100% (364/364), 56.16 MiB | 28.14 MiB/s, done.
Resolving deltas: 100% (200/200), done.


## Train a custom model

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/StyleGan_256.zip"

!unzip {zip_path} -d /content/drive/MyDrive/impressionist_landscapes_resized_1024/

Archive:  /content/drive/MyDrive/StyleGan_256.zip
   creating: /content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256/
  inflating: /content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256/a-y-jackson_a-copse-evening-1918_small.jpg  
  inflating: /content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256/a-y-jackson_early-spring-quebec-1923_small.jpg  
  inflating: /content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256/a-y-jackson_first-snow-algoma-country-1920_small.jpg  
  inflating: /content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256/a-y-jackson_frozen-lake-early-spring-algonquin-park-1914_small.jpg  
  inflating: /content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256/a-y-jackson_maple-woods-algoma-1920_small.jpg  
  inflating: /content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256/a-y-jackson_march-storm-georgian-bay-1920_small.jpg  
  inflating: /conten

### Convert dataset to .tfrecords

Next, we need to convert our image dataset to a format that StyleGAN2-ADA can read:`.tfrecords`.


In [ ]:
local_dataset_path = Path('/content/dataset1')
local_images_path =  Path('/content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256')

if (local_dataset_path).is_dir():
    print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
    print('Delete current dataset folder ({}) to regenerate tfrecords.'.format(local_dataset_path))
else:
    %mkdir "{local_dataset_path}"
    !python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images \
        "{local_dataset_path}" "{local_images_path}"

Loading images from "/content/drive/MyDrive/impressionist_landscapes_resized_1024/StyleGan_256"
Creating dataset "/content/dataset1"
/content/StyleGAN2-ADA/stylegan2-ada/dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 1000 images.


In [ ]:
!cp -r /content/dataset1/ /content/drive/MyDrive/

In [ ]:
!rm -r '/content/dataset1/'

In [ ]:
!python "{stylegan2_repo_path / 'train.py'}" --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [ ]:
training_path = Path('/content/drive/MyDrive/dataset1')
if not training_path.is_dir():
    %mkdir "{training_path}"

#how often should the model generate samples and a .pkl file
snapshot_count = 1
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = 'bgc'


resume_from = '/content/drive/MyDrive/dataset1/00000-dataset1-mirror-auto1-bgc-resumecustom/network-snapshot-000012.pkl'

!python "{stylegan2_repo_path / 'train.py'}" --outdir="/content/drive/MyDrive/dataset1" \
    --data="/content/drive/MyDrive/dataset1" \
    --snap={snapshot_count} --resume={resume_from} --augpipe={augs} \
    --mirror={mirrored} --mirrory={mirroredY} \
    --metrics={metric_list} #--dry-run

tcmalloc: large alloc 4294967296 bytes == 0x562d0f69c000 @  0x7fcb2c7c5001 0x7fcb299c854f 0x7fcb29a18b58 0x7fcb29a1cb17 0x7fcb29abb203 0x562d07ef9544 0x562d07ef9240 0x562d07f6d627 0x562d07f67ced 0x562d07efb48c 0x562d07f3c159 0x562d07f390a4 0x562d07ef9d49 0x562d07f6d94f 0x562d07f679ee 0x562d07e39e2b 0x562d07f69fe4 0x562d07f679ee 0x562d07e39e2b 0x562d07f69fe4 0x562d07f67ced 0x562d07e39e2b 0x562d07f69fe4 0x562d07efaafa 0x562d07f68915 0x562d07f679ee 0x562d07f676f3 0x562d080314c2 0x562d0803183d 0x562d080316e6 0x562d08009163
tcmalloc: large alloc 4294967296 bytes == 0x562e0f69c000 @  0x7fcb2c7c31e7 0x7fcb299c846e 0x7fcb29a18c7b 0x7fcb29a1935f 0x7fcb29abb103 0x562d07ef9544 0x562d07ef9240 0x562d07f6d627 0x562d07f679ee 0x562d07efabda 0x562d07f69737 0x562d07f679ee 0x562d07efabda 0x562d07f69737 0x562d07f679ee 0x562d07efabda 0x562d07f69737 0x562d07efaafa 0x562d07f68915 0x562d07f679ee 0x562d07efabda 0x562d07f6cd00 0x562d07f679ee 0x562d07efabda 0x562d07f69737 0x562d07f67ced 0x562d07efb48c 0x562d07f3

In [ ]:
!pip install contrib

In [6]:
!pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2301952/45929032 bytes (5.0%)5898240/45929032 bytes (12.8%)9134080/45929032 bytes (19.9%)12427264/45929032 bytes (27.1%)15613952/45929032 bytes (34.0%)19316736/45929032 bytes (42.1%)22503424/45929032 bytes (49.0%)25878528/45929032 bytes (56.3%)29360128/45929032 bytes (63.9%)32776192/45929032 bytes (71.4%)36421632/45929032 bytes (79.3%)39813120/45929032 bytes (86.7%)

In [7]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 6

generation_from = '/content/drive/MyDrive/dataset1/network-snapshot-000004.pkl'

!python "{stylegan2_repo_path / 'generate.py'}" generate-images \
    --outdir="{project_path / 'out'}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network={generation_from}

Loading networks from "/content/drive/MyDrive/dataset1/network-snapshot-000004.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Compiling... Loading... Done.
Generating image for seed 2974 (0/6) ...
Generating image for seed 2975 (1/6) ...
Generating image for seed 2976 (2/6) ...
Generating image for seed 2977 (3/6) ...
Generating image for seed 2978 (4/6) ...
Generating image for seed 2979 (5/6) ...
Generating image grid...


## Latent space exploration

In [8]:
from numpy import random
walk_types = ['line', 'sphere', 'noiseloop', 'circularloop']
latent_walk_path = project_path / 'out' / 'latent_walk'
if not latent_walk_path.is_dir():
    %mkdir "{latent_walk_path}"

explored_network = '//content/drive/MyDrive/dataset1/network-snapshot-000004.pkl'

seeds = [random.randint(10000) for i in range(10)]
print(','.join(map(str, seeds)))
print("Base seeds:", seeds)
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_path}" --trunc=0.7 --walk-type="{walk_types[2]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*20}

9491,8345,1810,3522,8491,1664,4957,1992,2204,200
Base seeds: [9491, 8345, 1810, 3522, 8491, 1664, 4957, 1992, 2204, 200]
Loading networks from "//content/drive/MyDrive/dataset1/network-snapshot-000004.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/200 ...
Generating image for step 1/200 ...
Generating image for step 2/200 ...
Generating image for step 3/200 ...
Generating image for step 4/200 ...
Generating image for step 5/200 ...
Generating image for step 6/200 ...
Generating image for step 7/200 ...
Generating image for step 8/200 ...
Generating image for step 9/200 ...
Generating image for step 10/200 ...
Generating image for step 11/200 ...
Generating image for step 12/200 ...
Generating image for step 13/200 ...
Generating image for step 14/200 ...
Generating image for step 15/200 ...
Generating image for step 16/200 ...
Generating image for step 17/200 ...
Gener